In [64]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn import preprocessing
import segyio
from random import randint
import os, os.path
import re 
%matplotlib

Using matplotlib backend: Qt5Agg


In [ ]:
plt.rcParams['figure.figsize'] = (2.56, 2.56)
plt.rcParams['figure.dpi'] = 129.95


In [ ]:
filenameA = '../../data/basic/dataA.segy'
filenameB = '../../data/basic/dataB.segy'

In [ ]:
seismic_A = segyio.tools.cube(filenameA)
# seismic_B = segyio.tools.cube(filenameB)
seismic_B = seismic_A

In [ ]:
print(seismic_A.shape==seismic_B.shape)

IL,XL,t=seismic_A.shape
print(IL,XL,t)

In [ ]:
# Gera pares de imagens aleatoreamente amostradas nos volumes sismicos para treinamento da GAN pix2pix
# loop percorrendo todas as inlines e salvando patches com o nome da IL e a posição do patche começando do canto superior esquerdo
# ILn--> InLine number ILp --> InLine patch

n_imagens=10
patch=256

for i in range(n_imagens):

    plt.rcParams['figure.figsize'] = (2.56, 2.56)
    plt.rcParams['figure.dpi'] = 129.95
    XLp=randint(0,(XL-patch))
    ILp=randint(0,(IL-patch))
    tp=randint(0,(t-patch))
    ILn=randint(0,IL-1)
    XLn=randint(0,XL-1)
    secaoIL=randint(0,1)
    
    if secaoIL==1:
#       Sismica A
        img=seismic_A[ILn,XLp:XLp+patch,tp:tp+patch]
        img_A = img.transpose()
        fig = plt.figure()
        plt.axis('off')
        plt.imshow(img_A,cmap='gray_r')
        name='./A/ILn'+str(ILn)+'XLp'+str(XLp)+'tp'+str(tp)+'.png'
        fig.savefig(name, bbox_inches='tight', pad_inches=0)
        plt.close(fig)  

#       Sismica B
        img=seismic_B[ILn,XLp:XLp+patch,tp:tp+patch]
        img_B = img.transpose()
        fig = plt.figure()
        plt.axis('off')
        plt.imshow(img_B,cmap='gray_r')
        name='./B/ILn'+str(ILn)+'XLp'+str(XLp)+'tp'+str(tp)+'.png'
        fig.savefig(name, bbox_inches='tight', pad_inches=0)
        plt.close(fig)  
        
#       Combina A e B 
        plt.rcParams['figure.figsize'] = (5.12, 2.56)
        img_AB = np.concatenate([img_A, img_B],1)
        fig = plt.figure()
        plt.axis('off')
        plt.imshow(img_AB,cmap='gray_r')
        name='./AB/ILn'+str(ILn)+'XLp'+str(XLp)+'tp'+str(tp)+'.png'
        fig.savefig(name, bbox_inches='tight', pad_inches=0)
        plt.close(fig)       


    else:
#       Sismica A
        img=seismic_A[ILp:ILp+patch,XLn,tp:tp+patch]
        img_A = img.transpose()
        fig = plt.figure()
        plt.axis('off')
        plt.imshow(img_A,cmap='gray_r')
        name='./A/XLn'+str(XLn)+'ILp'+str(ILp)+'tp'+str(tp)+'.png'
        fig.savefig(name, bbox_inches='tight', pad_inches=0)
        plt.close(fig)  
        
#       Sismica B        
        img=seismic_B[ILp:ILp+patch,XLn,tp:tp+patch]
        img_B = img.transpose()
        fig = plt.figure()
        plt.axis('off')
        plt.imshow(img_B,cmap='gray_r')
        name='./B/XLn'+str(XLn)+'ILp'+str(ILp)+'tp'+str(tp)+'.png'
        fig.savefig(name, bbox_inches='tight', pad_inches=0)
        plt.close(fig)  
        
#       Combina A e B 
        plt.rcParams['figure.figsize'] = (5.12, 2.56)
        img_AB = np.concatenate([img_A, img_B],1)
        fig = plt.figure()
        plt.axis('off')
        plt.imshow(img_AB,cmap='gray_r')
        name='./AB/XLn'+str(XLn)+'ILp'+str(ILp)+'tp'+str(tp)+'.png'
        fig.savefig(name, bbox_inches='tight', pad_inches=0)
        plt.close(fig)       


      

        
        




In [71]:
# Lê segy e gera figuras ordenadas para teste
# filename_test = '../../data/basic/dataA.segy'
# seismic_test = segyio.tools.cube(filename_test)
# seismic_scale=(seismic_test+32767.0)
seismic_scale=seismic_scale/seismic_scale.max()
IL,XL,t=seismic_test.shape
print(IL,XL,t,seismic_scale.mean(),seismic_scale.max(),seismic_scale.min())


651 951 462 0.499989 1.0 0.0


In [72]:
seismic_test=seismic_scale

In [73]:
# Escolha a IL:
ILn=300
p=50
npIL=XL//p #numero de patches por IL
npt=t//p #numero de patches por t
plt.rcParams['figure.figsize'] = (2.56, 2.56)
plt.rcParams['figure.dpi'] = 129.95
v0=0
v1=p
for i in range (npt):
    h0=0
    h1=p
    for k in range (npIL):
        img=seismic_test[ILn,h0:h1,v0:v1]
        img_test = img.transpose()
        fig = plt.figure()
        plt.axis('off')
        plt.imshow(img_test,cmap='gray_r')
        name='./test/ILn'+str(ILn)+'XLp'+str(i)+'tp'+str(k)+'.png'
        fig.savefig(name, bbox_inches='tight', pad_inches=0)
        plt.close(fig)  
        h0+=p
        h1+=p       
    v1+=p
    v0+=p   
   


In [74]:
# Reconstruindo a imagem a partir dos patches
p=256

IL_output= np.zeros([npt*p,npIL*p,4])

path = "./test/"
valid_images = [".jpg",".gif",".png",".tga"]
dire=os.listdir(path)
dire.sort()
x0=0
x1=0
t0=0
t1=0
for filename in dire:
    ext = os.path.splitext(filename)[1] 
    if ext.lower() in valid_images:
        numbers =re.findall('\d+',filename)
        XLp=int(numbers[1])
        tp=int(numbers[2])
        x0=(p*XLp)
        x1=p+(p*XLp)
        t0=(p*tp)
        t1=p+(p*tp)
        IL_output[x0:x1,t0:t1,:]=mpimg.imread(path+filename)
        

#         print(XLp,tp,x0,x1,t0,t1)
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['figure.dpi'] = 300.0
fig = plt.figure()
plt.axis('off')
plt.imshow(IL_output,cmap='gray_r')
name='./OUT/ILn_out'+str(ILn)+'.png'
fig.savefig(name, bbox_inches='tight', pad_inches=0)
plt.close(fig)          

In [ ]:
x=mpimg.imread(path+filename)
x.shape

In [20]:
import re 
numbers = re.findall('\d+',filename)

In [43]:
a=IL_output[2560:2816,0:256,:]
a.shape

(256, 256, 4)

In [49]:
numbers

['300']

In [63]:
mpimg.imread(path+filename)

array([[[0.4627451 , 0.4627451 , 0.4627451 , 1.        ],
        [0.4627451 , 0.4627451 , 0.4627451 , 1.        ],
        [0.4627451 , 0.4627451 , 0.4627451 , 1.        ],
        ...,
        [0.33333334, 0.33333334, 0.33333334, 1.        ],
        [0.33333334, 0.33333334, 0.33333334, 1.        ],
        [0.33333334, 0.33333334, 0.33333334, 1.        ]],

       [[0.4627451 , 0.4627451 , 0.4627451 , 1.        ],
        [0.4627451 , 0.4627451 , 0.4627451 , 1.        ],
        [0.4627451 , 0.4627451 , 0.4627451 , 1.        ],
        ...,
        [0.33333334, 0.33333334, 0.33333334, 1.        ],
        [0.33333334, 0.33333334, 0.33333334, 1.        ],
        [0.33333334, 0.33333334, 0.33333334, 1.        ]],

       [[0.4627451 , 0.4627451 , 0.4627451 , 1.        ],
        [0.4627451 , 0.4627451 , 0.4627451 , 1.        ],
        [0.4627451 , 0.4627451 , 0.4627451 , 1.        ],
        ...,
        [0.33333334, 0.33333334, 0.33333334, 1.        ],
        [0.33333334, 0.333333